In [1]:
import numpy as np
import pandas as pd

In [5]:
df =pd.read_csv('/content/diabetes.csv')

In [6]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72.0,35,169.5,33.6,0.627,50,1
1,1,85,66.0,29,102.5,26.6,0.351,31,0
2,8,183,64.0,32,169.5,23.3,0.672,32,1
3,1,89,66.0,23,94.0,28.1,0.167,21,0
4,0,137,40.0,35,168.0,43.1,2.288,33,1


In [7]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.495990
BloodPressure,0.174469
SkinThickness,0.295138
Insulin,0.377081
BMI,0.315577
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [8]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [71]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout


In [12]:
model = Sequential()
model.add(Dense(32, activation= 'relu', input_dim=8))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.7015 - loss: 0.6284 - val_accuracy: 0.7338 - val_loss: 0.5810
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7430 - loss: 0.5731 - val_accuracy: 0.7662 - val_loss: 0.5320
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7736 - loss: 0.5184 - val_accuracy: 0.7857 - val_loss: 0.5020
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7877 - loss: 0.4883 - val_accuracy: 0.7922 - val_loss: 0.4817
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7751 - loss: 0.4820 - val_accuracy: 0.7922 - val_loss: 0.4658
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8016 - loss: 0.4371 - val_accuracy: 0.7922 - val_loss: 0.4550
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8083 - loss: 0.4280 - val_accuracy: 0.8052 - val_loss: 0.4470
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8005 - loss: 0.4499 - val_accuracy: 0.8182 - 

In [17]:
import kerastuner as kt

<ipython-input-17-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [18]:
# 1. How to select appropriate optimizer
# 2. number of nodes in layer
# 3. how to select no. layers
# 4. All in one model

In [19]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation= 'relu', input_dim=8))
  model.add(Dense(1, activation = 'sigmoid'))

  optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop','adadelta'])
  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])
  return model

In [20]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 10s


In [22]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [23]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7735 - loss: 0.5404 - val_accuracy: 0.7987 - val_loss: 0.4845
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7855 - loss: 0.4871 - val_accuracy: 0.7857 - val_loss: 0.4660
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7860 - loss: 0.4747 - val_accuracy: 0.7727 - val_loss: 0.4543
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7790 - loss: 0.4631 - val_accuracy: 0.7727 - val_loss: 0.4481
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7726 - loss: 0.4700 - val_accuracy: 0.7727 - val_loss: 0.4430
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7905 - loss: 0.4473 - val_accuracy: 0.7792 - val_loss: 0.4397
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7707 - loss: 0.4528 - val_accuracy: 0.7792 - val_loss: 0.4372
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8111 - loss: 0.4322 - val_accuracy: 0.77

In [30]:
# this code for hyper tuning for best select nodes

def build_model(hp):
  model = Sequential()

  units = hp.Int('units', min_value=8, max_value=128, step=8)
  model.add(Dense(units=units, activation= 'relu', input_dim=8))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])
  return model

In [36]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    directory= 'my_dir',
    max_trials=5,
    project_name= 'KT'
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8246753215789795

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 14s


In [38]:
tuner.get_best_hyperparameters()[0].values

{'units': 128}

In [39]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [40]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7570 - loss: 0.4858 - val_accuracy: 0.7987 - val_loss: 0.4519
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8147 - loss: 0.4185 - val_accuracy: 0.7857 - val_loss: 0.4426
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7920 - loss: 0.4147 - val_accuracy: 0.7857 - val_loss: 0.4380
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7968 - loss: 0.4087 - val_accuracy: 0.7922 - val_loss: 0.4372
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8118 - loss: 0.3920 - val_accuracy: 0.7792 - val_loss: 0.4397
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7990 - loss: 0.4038 - val_accuracy: 0.7792 - val_loss: 0.4379
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8355 - loss: 0.3657 - val_accuracy: 0.7792 - val_loss: 0.4377
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8111 - loss: 0.3874 - val_accuracy: 0.78

In [41]:
# 3. how to select no. layers

In [47]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(72, activation= 'relu', input_dim=8))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(72, activation= 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics=['accuracy'])
  return model


In [48]:
tuner = kt.RandomSearch(build_model,
                        objective = kt.Objective('val_accuracy', direction='max'),
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'num_layres')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [49]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8376623392105103
Total elapsed time: 00h 00m 11s


In [50]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [52]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [55]:
model.fit(X_train, y_train, epochs = 100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.8623 - loss: 0.3268 - val_accuracy: 0.8442 - val_loss: 0.3916
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8782 - loss: 0.3311 - val_accuracy: 0.8506 - val_loss: 0.3927
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8975 - loss: 0.2731 - val_accuracy: 0.7468 - val_loss: 0.5773
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8986 - loss: 0.2540 - val_accuracy: 0.8182 - val_loss: 0.4946
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9057 - loss: 0.2663 - val_accuracy: 0.8312 - val_loss: 0.5551
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9131 - loss: 0.2280 - val_accuracy: 0.8182 - val_loss: 0.6205
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9032 - loss: 0.2287 - val_accuracy: 0.8442 - val_loss: 0.5157
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9355 - loss: 0.1652 - val_accuracy: 0.82

In [73]:
def build_model(hp):
  model = Sequential()

  counter = 0

  model.add(Dense(72, activation= 'relu', input_dim=8))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter==0:
      model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
                activation= hp.Choice('activation' + str(i), values= ['relu', 'tanh', 'sigmoid']),
                input_dim = 8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
                activation= hp.Choice('activation' + str(i), values= ['relu', 'tanh', 'sigmoid'])))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6,0.7,0.8,0.9])))
      counter+=1

  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer =hp.Choice('optimizer', values = ['rmsprop', 'adam,', 'sgd', 'nadam','adadelta']),
                loss = 'binary_crossentropy', metrics=['accuracy'])
  return model

In [74]:
tuner = kt.RandomSearch(build_model,
                        objective = kt.Objective('val_accuracy', direction='max'),
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'final1')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [75]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 17s


In [76]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 24,
 'activation0': 'relu',
 'dropout0': 0.8,
 'optimizer': 'nadam',
 'units1': 16,
 'activation1': 'relu',
 'dropout1': 0.7,
 'units2': 16,
 'activation2': 'sigmoid',
 'dropout2': 0.6,
 'units3': 112,
 'activation3': 'sigmoid',
 'dropout3': 0.7,
 'units4': 80,
 'activation4': 'tanh',
 'dropout4': 0.7,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.3,
 'units6': 40,
 'activation6': 'relu',
 'dropout6': 0.6,
 'units7': 112,
 'activation7': 'tanh',
 'dropout7': 0.1,
 'units8': 16,
 'activation8': 'relu',
 'dropout8': 0.1}

In [77]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 15 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [78]:
model.fit(X_train, y_train, epochs = 200, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7209 - loss: 0.5433 - val_accuracy: 0.8182 - val_loss: 0.4690
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7578 - loss: 0.5433 - val_accuracy: 0.8182 - val_loss: 0.4617
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7825 - loss: 0.4933 - val_accuracy: 0.8182 - val_loss: 0.4534
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7755 - loss: 0.4731 - val_accuracy: 0.8182 - val_loss: 0.4436
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7761 - loss: 0.4666 - val_accuracy: 0.8117 - val_loss: 0.4378
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7590 - loss: 0.4839 - val_accuracy: 0.8117 - val_loss: 0.4376
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8000 - loss: 0.4568 - val_accuracy: 0.8052 - val_loss: 0.4364
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7839 - loss: 0.4553 - val_accuracy: 0.81